In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

# List collections
print(client.get_collections())

# Check info collection
info = client.get_collection("chat_history")
print(info)


collections=[CollectionDescription(name='rag_knowledge'), CollectionDescription(name='chat_history')]
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=2 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_thr

In [5]:
from qdrant_client import QdrantClient
from pprint import pprint

client = QdrantClient(host="localhost", port=6333)

def inspect_qdrant():
    collections = client.get_collections().collections
    print(f"Found {len(collections)} collections:")
    
    for col in collections:
        print("\n" + "="*40)
        print(f"Collection name: {col.name}")
        try:
            info = client.get_collection(col.name)
            print(f"Points count: {info.points_count}")
            vectors = info.config.params.vectors
            if isinstance(vectors, dict):
                print("Vector names & sizes:")
                for name, vcfg in vectors.items():
                    print(f"  - {name}: size={vcfg.size}, distance={vcfg.distance}")
            else:
                print(f"Vector config: size={vectors.size}, distance={vectors.distance}")
        except Exception as e:
            print("Could not fetch collection info:", e)
        
        print("Sample points (up to 10):")
        try:
            resp, _ = client.scroll(col.name, limit=10, with_payload=True)
            points = getattr(resp, "result", resp)
            for p in points:
                vector_names = list(p.vector.keys()) if isinstance(p.vector, dict) else ["default"]
                print(f"  id={p.id}, vector_names={vector_names}, payload={p.payload}")
        except Exception as e:
            print("Could not fetch points:", e)

if __name__ == "__main__":
    inspect_qdrant()


Found 1 collections:

Collection name: collection
Points count: 0
Vector config: size=384, distance=Cosine
Sample points (up to 10):


In [19]:
from qdrant_client import QdrantClient

# Kết nối Qdrant (ví dụ chạy local)
client = QdrantClient("http://localhost:6333")

collection_name = "collection"

# Lấy thông tin collection
info = client.get_collection(collection_name)
print("Tổng số vector:", info.vectors_count)


Tổng số vector: None


In [7]:
import numpy as np

# Giả sử bạn có embedding vector test 
dummy_vector = np.random.rand(384).tolist()

result = client.search(
    collection_name=collection_name,
    query_vector=dummy_vector,
    limit=3
)

print("Kết quả tìm được:", result)


Kết quả tìm được: []


C:\Users\admin\AppData\Local\Temp\ipykernel_14208\2745754574.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = client.search(


In [2]:
import qdrant_client

def check_collections():
    client = qdrant_client.QdrantClient(host="localhost", port=6333)

    collections = client.get_collections()
    print("📂 Danh sách collections trong Qdrant:")
    for coll in collections.collections:
        name = coll.name

        # Dùng count để chắc chắn có số vector
        count = client.count(collection_name=name, exact=True).count
        print(f" - {name}: {count} vectors")

    if not collections.collections:
        print("⚠️ Không có collection nào trong Qdrant (rỗng).")

if __name__ == "__main__":
    check_collections()


📂 Danh sách collections trong Qdrant:
 - collection: 0 vectors


In [14]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

# Xem các collection hiện có
print(client.get_collections())

# Nếu có collection "documents", xem thử số lượng points
print(client.count(collection_name="collection", exact=True))

# Lấy vài point mẫu
points = client.scroll(collection_name="collection", limit=5)
print(points)


collections=[CollectionDescription(name='collection')]
count=1
([Record(id=351531, payload={'source': 'app'}, vector=None, shard_key=None, order_value=None)], None)


In [5]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://cafef.vn"


def get_url(page: int) -> str:
    return f"{BASE_URL}/doc-nhanh.chn" if page == 1 else f"{BASE_URL}/doc-nhanh/trang-{page}.chn"


def get_article_content(link: str) -> str:
    """Vào trang chi tiết và lấy toàn bộ nội dung"""
    resp = requests.get(link, timeout=10)
    resp.encoding = "utf-8"
    soup = BeautifulSoup(resp.text, "html.parser")

    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    # Ghép các đoạn văn lại
    paragraphs = [p.get_text(strip=True) for p in content_div.find_all("p") if p.get_text(strip=True)]
    return "\n".join(paragraphs)


def crawl_cafef(max_pages: int = 1):
    articles = []
    for page in range(1, max_pages + 1):
        url = get_url(page)
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        blocks = soup.select("div.nv-text-cont")
        for block in blocks:
            link_tag = block.select_one("a.news-title")
            if not link_tag:
                continue

            href = link_tag["href"]
            article_id = href.split("-")[-1].replace(".chn", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href

            details = block.find_next_sibling("div", class_="nv-details")
            short_content = details.select_one("div.abs").text.strip() if details else ""
            time_tag = details.select_one("span.time")
            time_text = time_tag.get_text(strip=True) if time_tag else ""

            # Lấy toàn bộ nội dung trong link
            full_content = get_article_content(link)

            articles.append({
                "id": article_id,
                "title": title,
                "time": time_text,
                "summary": short_content,
                "link": link,
                "content": full_content
            })
    return articles


if __name__ == "__main__":
    data = crawl_cafef(max_pages=2)
    for item in data[:2]:
        print(item)


{'id': '188250923075226457', 'title': 'Vụ khách hàng kiện công ty bảo hiểm Manulife: Tòa không chấp nhận toàn bộ yêu cầu khởi kiện của nguyên đơn', 'time': '(23/09/2025 08:03)', 'summary': 'Toà cho rằng việc khách hàng khởi kiện Manulife với cáo buộc "lừa dối trong việc giao kết hợp đồng bảo hiểm" là không có căn cứ.', 'link': 'https://cafef.vn/vu-khach-hang-kien-cong-ty-bao-hiem-manulife-toa-khong-chap-nhan-toan-bo-yeu-cau-khoi-kien-cua-nguyen-don-188250923075226457.chn', 'content': 'Ngày 22/9, Tòa án nhân dân Khu vực 5 - Thành phố Hồ Chí Minh mở phiên tòa xét xử vụ kiện giữa bà Đ.P.D.L với công ty TNHH Manulife (Việt Nam), liên quan đến tranh chấp hợp đồng bảo hiểm nhân thọ. Bà L. ủy quyền cho luật sư tham gia phiên tòa.\nTheo đơn khởi kiện, bà L. cho rằng đã được nhân viên ngân hàng SCB giới thiệu tham gia chương trình tiết kiệm đầu tư lãi suất cao có kèm bảo hiểm miễn phí. Khi làm hồ sơ tham gia chương trình, bà L. được biết đây là hợp đồng bảo hiểm nhân thọ. Bà L. cho rằng có nhiề

In [7]:
from qdrant_client import QdrantClient
client = QdrantClient()
print(client.get_collections())


collections=[CollectionDescription(name='collection')]


In [1]:
from qdrant_client import QdrantClient
import os

self.client = QdrantClient(
    host=os.getenv("QDRANT_HOST", "qdrant"),
    port=int(os.getenv("QDRANT_PORT", 6333))
)


NameError: name 'self' is not defined

In [17]:
from qdrant_client import QdrantClient

def list_vectors_in_qdrant(collection_name: str = "my_collection", host: str = "localhost", port: int = 6333, limit: int = 100):
    """
    Liệt kê tất cả vectors trong collection và in ID + payload + độ dài vector.
    """
    client = QdrantClient(host=host, port=port)

    try:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=limit
        )

        if not points:
            print(f"Collection '{collection_name}' hiện chưa có vector nào.")
        else:
            print(f"Collection '{collection_name}' có {len(points)} vector (lấy tối đa {limit}):")
            for point in points:
                print(f"ID: {point.id}, payload: {point.payload}, vector length: {len(point.vector) if point.vector else 0}")

            # nếu còn next_page thì nghĩa là collection còn nhiều vector nữa
            if next_page:
                print("⚠️ Collection còn vector khác, hãy tăng limit hoặc tiếp tục scroll với offset.")
    except Exception as e:
        print("Lỗi khi truy vấn Qdrant:", e)

# Test
list_vectors_in_qdrant(collection_name="collection")


Collection 'collection' có 12 vector (lấy tối đa 100):
ID: 86651, payload: {'source': 'app'}, vector length: 0
ID: 175468, payload: {'source': 'app'}, vector length: 0
ID: 190315, payload: {'source': 'app'}, vector length: 0
ID: 236817, payload: {'source': 'app'}, vector length: 0
ID: 318901, payload: {'source': 'app'}, vector length: 0
ID: 351531, payload: {'source': 'app'}, vector length: 0
ID: 462858, payload: {'source': 'app'}, vector length: 0
ID: 534956, payload: {'source': 'app'}, vector length: 0
ID: 586207, payload: {'source': 'app'}, vector length: 0
ID: 603741, payload: {'source': 'app'}, vector length: 0
ID: 798773, payload: {'source': 'app'}, vector length: 0
ID: 845293, payload: {'source': 'app'}, vector length: 0
